![TextRank](https://miro.medium.com/max/1050/1*JeYlqJTA5MpKsaWwHkhKQw.jpeg)
https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0

# 有向无权图

将文本中的词语作为节点（顶点集$V$），词语之间的邻接关系作为边（边集$E$，其中$E$是$V\times V$的子集），构成有向图，即词图。词图本质上为一个二元组。对于给定的顶点$V{i}$，$\operatorname{In}\left(V_{i}\right)$是指向它的顶点集（前置），而$Out\left(V_{i}\right)$ 是顶点$V{i}$指向的顶点集（后继）。$TextRank$是词图当中度量词语重要性的主要方法，该方法以迭代方式按照以下公式计算节点 $v_i$的重要性：

$$
S\left(V_{i}\right)=(1-d)+d * \sum_{j \in I n\left(V_{i}\right)} \frac{1}{\left|O u t\left(V_{j}\right)\right|} S\left(V_{j}\right)
$$

其中，$S (v_j)$表示节点 $v_j$的重要性得分；$d$ 为阻尼系数(damping factor)，通常取值为 0.85，在公式中表示任意时刻都有 $1 - d$ 的概率由当前节点随机跳转到词图中的其他节点之上。当计算收敛之后，$S (v_i) $值可以表示节点$ v_i$在词图中的重要性。

<img src="Graph.png" width="300" height="100">

图中这个例子，Vi的重要性由指向它的Vj（V1和V2）的重要性决定，同时分母要平衡这些节点跳转到其他节点的可能性。

阻尼系统代表异常情况，不按照我们的模型走。

$$
S\left(V_{i}\right)=(1-d)+d *(\frac{1}{\left|Out\left(V_{1}\right)\right|} S\left(V_{1}\right))+\frac{1}{\left|Out\left(V_{2}\right)\right|} S\left(V_{2}\right)))
$$

其中，${\left|Out\left(V_{1}\right)\right|} = 2$, ${\left|Out\left(V_{2}\right)\right|} = 3$


# 有向带权图

不同于PageRank，TextRank是自然语言文本，存在更复杂的内部结构，因此可能需要将两个顶点之间连接的“强度”纳入模型。

因此带权的重要性值表示为：

$$
W S\left(V_{i}\right)=(1-d)+d * \sum_{V_{j} \in I n\left(V_{i}\right)} \frac{w_{j i}}{\sum_{V_{k} \in \operatorname{Out}\left(V_{j}\right)} w_{j k}} W S\left(V_{j}\right)
$$

除了词语，各种大小和特征的文本单元可以添加为图形中的顶点，例如单词、短语、句子甚至篇章，并且在任意两个顶点之间绘制连接的关系类型，例如词汇或语义关系、上下文重叠等。

<img src="Graph2.png" width="300" height="100">

# 算法主要步骤

基于图的排序算法在自然语言文本中的应用包括以下主要步骤：

1.确定最能定义手头任务的文本单元，并将它们作为图中的顶点。

2.确定连接这些文本单元的关系，并使用这些关系在图中的顶点之间绘制边。边可以是有向的或无向的、加权的或无权的。

3.迭代基于图的链接算法直到收敛。

4.根据它们最后的得分排序并指导决策。

# 代码库

In [9]:
text = """Automatic summarization is the process of reducing a text document with a \
computer program in order to create a summary that retains the most important points \
of the original document. As the problem of information overload has grown, and as \
the quantity of data has increased, so has interest in automatic summarization. \
Technologies that can make a coherent summary take into account variables such as \
length, writing style and syntax. An example of the use of summarization technology \
is search engines such as Google. Document summarization is another."""

In [6]:
from summa import summarizer
print(summarizer.summarize(text))

Automatic summarization is the process of reducing a text document with a computer program in order to create a summary that retains the most important points of the original document.


In [10]:
from summa import keywords
print(keywords.keywords(text))

document
automatic summarization
technologies
technology


In [ ]:
import spacy
import pytextrank

# example text
text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

doc = nlp(text)

In [15]:
# examine the top-ranked phrases in the document
for p in doc._.phrases[:3]:
    #print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
    print(p.chunks[0])

minimal generating sets
systems
linear Diophantine equations


In [28]:
import pandas as pd
df = pd.read_excel('example.xlsx')
text = df['标题'].to_string(index=False)

In [42]:
df = pd.read_csv('https://raw.githubusercontent.com/iamxiatian/keyphrase/master/data/paper_abstract.csv')

In [51]:
text = df['摘要'][1]

In [52]:
import codecs
from textrank4zh import TextRank4Keyword, TextRank4Sentence

#text = codecs.open('01.txt', 'r', 'utf-8').read()
tr4w = TextRank4Keyword()

tr4w.analyze(text=text, lower=True, window=2)  # py2中text必须是utf8编码的str或者unicode对象，py3中必须是utf8编码的bytes或者str对象

In [53]:
df['关键词'][1]

'高校图书馆;3D打印服务;迈阿密大学;B.E.S.T.图书馆;美国;探索;启示'

In [54]:
df['摘要'][1]

'迈阿密大学B.E.S.T.图书馆作为美国较早提供3D打印服务的机构,其对3D打印服务的探索具有先创性。本文从满足用户创意实践需求、广泛吸收参与伙伴、提供全方位服务保障3个方面阐述了B.E.S.T.图书馆3D打印服务的创新性实践探索内容,并进一步探讨了我国高校图书馆3D打印服务可以借鉴的经验,对提升我国高校图书馆3D打印服务效能具有一定指导价值。'

In [55]:
print( '关键词：' )
for item in tr4w.get_keywords(30, word_min_len=1):
    print(item.word, item.weight)

关键词：
打印服务 0.07651220752870871
实践 0.06427901995062724
探索 0.061599170287937585
图书馆 0.05316577243229686
参与 0.047294922187070584
伙伴 0.047294922187070584
经验 0.039202273522620776
创意 0.03721203985987201
服务 0.03642987249544627
保障 0.03642987249544627
方面 0.03642987249544627
阐述 0.03642987249544627
指导 0.03642987249544627
价值 0.03642987249544627
借鉴 0.03513358410646303
具有 0.03222469114592543
效能 0.03216811039052347
创新性 0.03213190804673634
美国 0.028061922527020965
高校 0.028061922527020965
提供 0.02556482280382194
吸收 0.02556482280382194
提升 0.022125978477283224
满足用户 0.02127906821748389
需求 0.019124302417556527
内容 0.018553774481012753
机构 0.01847208650349675
大学 0.005464480874316941
创性 0.005464480874316941
探讨 0.005464480874316941


In [56]:
print( '关键短语：' )
for phrase in tr4w.get_keyphrases(keywords_num=20, min_occur_num= 2):
    print(phrase)

关键短语：
高校图书馆
